In [1]:
!pip install requests

In [2]:
!pip install pandas

In [3]:
import requests
import json
import pandas as pd
retList = []


In [4]:
#호출용 파라미터 지정
def getReviewParam(place, theme, pageNum = 1):
    
    
    return {
    "operationName": "getVisitorReviews",
    "variables": {
      "input": {
        "businessId": place['id'],
        "businessType": "restaurant",
        "item": "0",
        "page": pageNum,
        "size": 40,
        "includeContent": True,
        "getUserStats": True,
        "includeReceiptPhotos": True,
        "cidList": place['categoryPath'][0],
        "theme": theme
      },
      "id": place['id']
    },
  "query": "query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        class\n        videoId\n        videoOriginSource\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        replyTitle\n        isReported\n        isSuspended\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      showBookingItemOptions\n      bookingItemName\n      bookingItemOptions\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      isFollowing\n      followerCount\n      followRequested\n      loginIdno\n      receiptInfoUrl\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
  }



#카테고리 현황 조회용 파라미터
def getStatParam(place):
    return {
  "operationName": "getVisitorReviewStats",
  "variables": {
    "businessType": "restaurant",
    "id": place['id'],
    "itemId": "0"
  },
  "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
}



# {
#   "operationName": "getVisitorReviewStats",
#   "variables": {
#     "businessType": "restaurant",
#     "id": "1813392509",
#     "itemId": "0"
#   },
#   "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
# }




def placeProc(place):
    #장소별 ret 초기화
    placeDict = {}

    placeDict['id'] =  place['id']
    placeDict['name'] =  place['name']
    placeDict['contents'] =  []
    
    statParam = getStatParam(place)

    # 리뷰/테마를 읽어오기 위한 api 정의
    review_url = "https://pcmap-api.place.naver.com/graphql"
    review_headers = {
        "referer": "https://pcmap.place.naver.com/",
        "content-type": "application/json",
        "accept": "*/*",
        "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
    }

    # 테마 목록 가져오기 
    stat_response = requests.post(review_url, headers=review_headers, data=json.dumps(statParam))
    print('stat request', place['name'], stat_response)
    if stat_response.status_code != 200:        
        return placeDict;
    stat_json = json.loads(stat_response.text)

    # 테막 목록 확인
    themes = stat_json['data']['visitorReviewStats']['analysis']['themes']
    # 테마별 리뷰를 불러온다.
    placeDict['contents'] = []
    for theme in themes:
        # total은 제외한다
        if theme['code'] in ['taste','service','price']:
            nextPage = True
            pageNum = 1
            print (pageNum, nextPage)
            while nextPage:
              print (pageNum, nextPage)
              review_param = getReviewParam(place,theme['code'], pageNum)
              review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
              if review_response.status_code != 200: 
                  continue;
              review_json = json.loads(review_response.text)
              reviewBody = review_json['data']['visitorReviews']['items']
              print(theme['code'], len(reviewBody))
              placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))
              if len(reviewBody)==40:
                  pageNum = pageNum + 1
              else:
                  nextPage = False
                  break
            # #반환된 review 건수가 40건인 경우 page 2 한번 더 호출한다.
            # if len(reviewBody)==40:
            #     review_param = getReviewParam(place, theme['code'], 2)
            #     review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
            #     if review_response.status_code != 200: 
            #         continue;
            #     review_json = json.loads(review_response.text)
            #     reviewBody = review_json['data']['visitorReviews']['items']
            #     print(theme['code'], len(reviewBody))
            #     placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))

    return placeDict
    
    

In [13]:
url = 'https://map.naver.com/v5/api/search'
placeName="sunuri"

headers = {
    'authority': 'map.naver.com',
    'content-type': 'application/json',
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
}

### 음식점 목록을 가져온다 (현재 위치 위경도값:searchCoord, query : 검색어)
### 호출건수 : displayCount : 100
params = {
    'lang': 'ko',
    'caller': 'pcweb',
    'type': 'place',
    #'searchCoord': '37.272429000000116,127.13596284980378', #용인
    'query': '수누리감자탕 영통점',
    'page': 1,
    'displayCount': 1,
    'isPlaceRecommendationReplace': 'false',
}


# clientX: 127.1275729
# clientY: 37.272429
# x: 127.10589175393739
# y: 37.36958999999972
# bounds: 127.09966902902255;37.363723468708145;127.12069754769959;37.37528555368495


In [14]:

response = requests.get(url, headers=headers, params=params)

result = json.loads(response.text)

#호출결과에서 장소에 대한 목록만 읽어온다
placeList = result['result']['place']['list']

In [15]:
len(placeList)

1

In [16]:
print(placeList)

[{'index': '0', 'rank': '1', 'id': '18258721', 'name': '수누리감자탕 영통점', 'tel': '031-202-8441', 'isCallLink': False, 'virtualTel': '', 'virtualTelDisplay': '', 'ppc': '1', 'category': ['한식', '감자탕'], 'categoryPath': [['220036', '220037', '220067', '230825']], 'rcode': '02117105', 'businessStatus': {'requestTime': '202306100208', 'status': {'code': 8, 'text': '24시간 영업', 'emphasis': False, 'description': '24시간 영업', 'detailInfo': ''}, 'businessHours': '202306100000~202306102400', 'breakTime': '', 'lastOrder': ''}, 'naviInfoText': None, 'address': '경기도 수원시 영통구 영통동 1024-2 경희대 정문 앞', 'roadAddress': '경기도 수원시 영통구 덕영대로 1691 경희대 정문 앞', 'abbrAddress': '영통동 1024-2 경희대 정문 앞', 'shortAddress': ['경기 수원시 영통구', '덕영대로 1691', '경희대 정문 앞'], 'display': '<b>수누리감자탕 영통점</b>', 'telDisplay': '031-202-8441', 'context': ['감자탕', '시래기', '해물찜', '재방문', '아이스크림'], 'reviewCount': 226, 'placeReviewCount': 1611, 'ktCallMd': 'f8f5e426d518dde091d22d1b4980cc6d', 'coupon': '0', 'thumUrl': 'https://ldb-phinf.pstatic.net/20200811_103/

In [17]:
# 길이 확인 및 첫 음식점 확인
print(len(placeList))
retList = []
placeList[0]
print(placeList[0]['id'])
cnt = 0
for place in placeList:
    cnt+=1
    # if cnt<50: continue
    retList.append(placeProc(place))
    
    
# retList.append(placeProc(placeList[1]))
# retList.append(placeProc(placeList[2]))

1
18258721
stat request 수누리감자탕 영통점 <Response [200]>
1 True
1 True
taste 40
2 True
taste 40
3 True
taste 40
4 True
taste 40
5 True
taste 40
6 True
taste 40
7 True
taste 40
8 True
taste 8
1 True
1 True
service 24
1 True
1 True
price 3


In [31]:
cnt = 0
last_json = []
for place in retList:
    if cnt>1: continue;
    if len(place['contents'])>0:
        # 'div' 키의 값이 'taste', 'service', 'price' 인 데이터만 저장할 리스트
        last_json.extend(list(filter(lambda x: x.get('div') in ['taste', 'service', 'price'], place['contents'])))

    

In [32]:
last_json

[{'div': 'taste', 'cont': '자주는 못가지만 영통갈때마다 들려요\n너무맛있음 ㅠ 주차장이 좀 더 넓으면 얼마나 좋으꼬'},
 {'div': 'taste',
  'cont': '감자탕에 뼈 양도 많고 살코기도 많이 붙어있어요!!! 당면이랑 떡사리 등 사리 종류가 다양해요\n근데 볶음밥은 별로에여..'},
 {'div': 'taste', 'cont': '친구랑 한뚝빼기 하고 왔어요. 맛나네요^^'},
 {'div': 'taste',
  'cont': '세상에 가성비 미쳤어요ㅋㅋ학교 앞에서 성공하는데는 다 이유가 있나봐여!! 저희는 직장인들끼리 모임을 했는데 진짜 배터지게 먹고 술도 마시고 했는데 계산할 때 어케 돈이 이거밖에 안 나왔지? 했어요ㅋㅋㅋ국물 존맛탱이구 라면사리 추가는 진짜 필수인듯♡ 사장님 번창하세요😆'},
 {'div': 'taste', 'cont': '뼈해장국 굉장히 맛있네요!'},
 {'div': 'taste', 'cont': '잡내없고, 맛있어요!\n칼칼하게 입을 당기는 맛!!'},
 {'div': 'taste', 'cont': '양도많고,맛있어요'},
 {'div': 'taste', 'cont': '아들 추천으로 \n맛있게 먹고왔어요~♥️\n찐 맛집이네요\n재방문 할게요 감사합니다🎶'},
 {'div': 'taste',
  'cont': '양이 많고 가성비가 좋아요. 기대했으나 맛은 보통.맵네요. 김치는 맛있어요.  근데 직원들이 불친절하고 퉁명스러워요. 특히 나이 많으신 아주머니.. 벨은 고장나서 안 눌리고 직원들은 불러도 몸은 오지도 않고 다들 그자리에서 쳐다보면서 대답하시네요. 부엌 쪽에서 슥 보시면서 뭐 필요하냐고 물어봐서 당황했어요ㅋ 그런 것 때문에 기분 상할 일이 이것저것 많을 것 같은 가게네요'},
 {'div': 'taste', 'cont': '맛있고 항상 손님이 북적한 집'},
 {'div': 'taste', 'cont': '맛있어요 샤람이많으네요^^'},
 {'div': 'taste',
  'cont': 

In [33]:
df = pd.DataFrame(last_json)

In [34]:
df

,div,cont
0,taste,자주는 못가지만 영통갈때마다 들려요\n너무맛있음 ㅠ 주차장이 좀 더 넓으면 얼마나 좋으꼬
1,taste,감자탕에 뼈 양도 많고 살코기도 많이 붙어있어요!!! 당면이랑 떡사리 등 사리 종류...
2,taste,친구랑 한뚝빼기 하고 왔어요. 맛나네요^^
3,taste,세상에 가성비 미쳤어요ㅋㅋ학교 앞에서 성공하는데는 다 이유가 있나봐여!! 저희는 직...
4,taste,뼈해장국 굉장히 맛있네요!
...,...,...
310,service,사장님이 친절하세요~
311,service,푸짐하고 맛있어요ㆍㆍ친절한 직원분들때문이라도 자주들리고싶네요^^
312,price,음식종류가 많고 가격이 무척 저렴해요
313,price,가성비좋은 괜찮은집


In [79]:
df.to_csv('sunuri.csv')